In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from itertools import product

import lightgbm as lgb
from tqdm import tqdm_notebook

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
test

In [ ]:
oct_trans=sales_train[sales_train["date"].str.endswith("10.2015")]
oct_sales = oct_trans.groupby(['shop_id','item_id'])['item_cnt_day'].sum()

In [ ]:
nov_trans=sales_train[sales_train["date"].str.endswith("11.2014")]
nov_sales = nov_trans.groupby(['shop_id','item_id'])['item_cnt_day'].sum()

In [ ]:
merged=pd.merge(test,oct_sales,on=['shop_id','item_id'],how="outer")
#merged=merged[merged['ID']!=np.nan]
merged=merged.dropna(subset=['ID'])
merged=merged.replace(np.nan,0)
merged['item_cnt_month']=merged['item_cnt_day'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id','item_cnt_day'],axis=1)
merged['ID']=merged['ID'].astype('int')
merged.to_csv("october_sales.csv",index=False)
merged
# print(merged.shape)
# sample_submission.shape

In [ ]:
sales = sales_train

# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
all_data2 = all_data.copy()
del grid, gb 
gc.collect();

In [ ]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

In [ ]:
print(all_data)

In [ ]:
dates = all_data['date_block_num']

last_block = dates.max()
print(last_block)
X_train = all_data.loc[dates <=  last_block].drop(to_drop_cols, axis=1)
#print(X_train.head)
#X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)
#print(X_test.head)

y_train = all_data.loc[dates <=  last_block, 'target'].values
#y_test =  all_data.loc[dates == last_block, 'target'].values

In [ ]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data2.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

test_data = test.copy()
test_data['date_block_num'] = 34

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data2[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    test_data = pd.merge(test_data, train_shift, on=index_cols, how='left').fillna(0)

# # Don't use old data from year 2013
#all_data = all_data[all_data['date_block_num'] >= 12] 

# # List of all lagged features
# fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# # We will drop these at fitting stage
# to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# # Category for each item
# item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

test_data = pd.merge(test_data, item_category_mapping, how='left', on='item_id')
test_data = downcast_dtypes(test_data)
gc.collect();

# print(test_data.head)


In [ ]:
print(X_train.size)

In [ ]:
y_train = y_train.clip(0,20)

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)




In [ ]:
# y_train = y_train.clip(0,20)
# print(np.max(y_train))

In [ ]:
to_drop_cols = ['ID','date_block_num']
X_test =  test_data.drop(to_drop_cols, axis=1)
# print(X_train.columns)
# print(X_test.columns)
pred_lgb = model.predict(X_test)
#print(pred_lgb)

merged = test.copy()
merged['item_cnt_month'] = pred_lgb
merged['item_cnt_month']=merged['item_cnt_month'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id'],axis=1)
merged['ID']=merged['ID'].astype('int')
merged.to_csv("lightgbm_lag_trainclipped.csv",index=False)
print(merged.head)